In [1]:
import pandas as pd
import numpy as np
import json
import os
import networkx as nx
import pickle5 as pickle

from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.preprocessing import LabelEncoder

from myclass.CleanMergeDataset import Clean_Merge_Dataset
from myclass.BonferroniTtest import Bonferroni_Ttest

In [13]:
if os.path.exists('final_dataset_common.json') is False:

    data_normal = pd.read_pickle('./data-ready/RNA_dataframe_normal').replace('/', '\\')
    data_tumor = pd.read_pickle('./data-ready/RNA_dataframe').replace('/', '\\')
    dataset_RNA, y_RNA, cases_id_RNA = Clean_Merge_Dataset(name='RNA').transform(data_normal, data_tumor)
    df_RNA = pd.concat([dataset_RNA, cases_id_RNA], axis=1)

    data_normal = pd.read_pickle('./data-ready/miRNA_dataframe_normal').replace('/', '\\')
    data_tumor = pd.read_pickle('./data-ready/miRNA_dataframe').replace('/', '\\')
    dataset_miRNA, y_miRNA, cases_id_miRNA= Clean_Merge_Dataset(name='miRNA').transform(data_normal, data_tumor)
    df_miRNA = pd.concat([dataset_miRNA, cases_id_miRNA], axis=1)

    data_normal = pd.read_pickle('./data-ready/illumina-27-450-normal').replace('/', '\\')
    data_tumor = pd.read_pickle('./data-ready/illumina450-27-tumor').replace('/', '\\')
    dataset_illumina, y_illumina, cases_id_illumina= Clean_Merge_Dataset(name='illumina').transform(data_normal, data_tumor)
    df_illumina = pd.concat([dataset_illumina, cases_id_illumina], axis=1)

    dataset_RNA = Bonferroni_Ttest(label_case_id_into_X=True, alpha=0.05).fit_transform(pd.concat([df_RNA, y_RNA], axis=1), y_RNA)
    dataset_miRNA = Bonferroni_Ttest(label_case_id_into_X=True, alpha=0.05).fit_transform(pd.concat([df_miRNA, y_miRNA], axis=1), y_miRNA)
    dataset_illumina = Bonferroni_Ttest(label_case_id_into_X=True, alpha=0.05).fit_transform(pd.concat([df_illumina, y_illumina], axis=1), y_illumina)

    cases_id = set(dataset_illumina['case_id']) & set(dataset_miRNA['case_id']) & set(dataset_RNA['case_id'])
    df_final_illumina = dataset_illumina.loc[dataset_illumina['case_id'].isin(cases_id)]
    df_final_rna = dataset_RNA.loc[dataset_RNA['case_id'].isin(cases_id)]
    df_final_mirna = dataset_miRNA.loc[dataset_miRNA['case_id'].isin(cases_id)]

    print(df_final_illumina.shape)
    print(df_final_rna.shape)
    print(df_final_mirna.shape)
    
    df_final_illumina.to_pickle('illumina_pickle.pkl')
    df_final_rna.to_pickle('rna_pickle.pkl')
    df_final_mirna.to_pickle('miRNA_pickle.pkl')
    
    my_dict = {
        'miRNA': df_final_mirna.to_dict(),
        'RNA': df_final_rna.to_dict(),
        'illumina': df_final_illumina.to_dict()
    }
    with open('final_dataset_common.json', 'w') as outfile:
        json.dump(my_dict, outfile)
    
    df_illumina = df_final_illumina.copy()
    df_mirna = df_final_mirna.copy()
    df_rna = df_final_rna.copy()
    
    del my_dict
    del df_final_illumina
    del df_final_rna
    del df_final_mirna
    del dataset_illumina
    del dataset_RNA
    del dataset_miRNA
else:
    #df_illumina = pd.read_pickle('illumina_pickle.pkl')
    #df_mirna = pd.read_pickle('miRNA_pickle.pkl')
    #df_rna = pd.read_pickle('rna_pickle.pkl')
    with open('illumina_pickle.pkl' ,'rb') as f:
        df_illumina = pickle.load(f)
    with open('mirna_pickle.pkl' ,'rb') as f:
        df_mirna = pickle.load(f)
    with open('rna_pickle.pkl' ,'rb') as f:
        df_rna = pickle.load(f)

In [22]:
import numpy as np
import pandas as pd

from scipy.spatial.distance import pdist, squareform
from copy import deepcopy
from sklearn.preprocessing import StandardScaler

class SimilarityNetworkFusion:
    def __init__(self, df_mirna, df_rna, df_illumina, k=3, mu=0.3):
        
        self.cases_id = df_rna.loc[:, 'case_id']
        self.rna = df_rna.copy()
        self.mirna = df_mirna.copy()
        self.illumina = df_illumina.copy()
        
        self.k = k
        self.mu = mu
        self.check_columns()
    
    def calculate_matrix(self):
        if hasattr(self, 'w_rna') is False:
            self.w_rna = self.__weights__(self.rna, 'RNA')
            self.w_mirna = self.__weights__(self.mirna, 'miRNA')
            self.w_illumina = self.__weights__(self.illumina, 'Illumina')
        
        if hasattr(self, 'p_rna') is False:
            self.starting_p_rna = self.P_matrix(self.w_rna.to_numpy().tolist(), self.cases_id.shape[0], 'RNA')
            self.starting_p_mirna = self.P_matrix(self.w_mirna.to_numpy().tolist(), self.cases_id.shape[0], 'miRNA')
            self.starting_p_illumina = self.P_matrix(self.w_illumina.to_numpy().tolist(), self.cases_id.shape[0], 'Illumina')

        self.s_rna = self.S_matrix(self.w_rna.to_numpy().tolist(), self.cases_id.shape[0], 'RNA')
        self.s_mirna = self.S_matrix(self.w_mirna.to_numpy().tolist(), self.cases_id.shape[0], 'miRNA')
        self.s_illumina = self.S_matrix(self.w_illumina.to_numpy().tolist(), self.cases_id.shape[0], 'Illumina')
        
        return self
        

    def __weights__(self, dataset, name):
        print('Calculating weights for {}...'.format(name))
        df = pd.DataFrame(columns=self.cases_id, data=dataset.T.values)
        
        #calculate euclidean distance
        dist = pdist(dataset, 'euclidean')
        self.df_dist = pd.DataFrame(columns=self.cases_id, index=self.cases_id, data=squareform(dist))
        weights = pd.DataFrame(columns=self.cases_id, index=self.cases_id, data=[])
                
        for i, patient_i in enumerate(tqdm(self.cases_id)):
            for patient_j in self.cases_id.iloc[i:]:
                    tokK_mean_i = np.sort(self.df_dist.loc[patient_i, :].to_numpy())[:self.k].mean()
                    topK_mean_j = np.sort(self.df_dist.loc[patient_j, :].to_numpy())[:self.k].mean()
                    
                    eps = (tokK_mean_i + tokK_mean_i + self.df_dist.loc[patient_i, patient_j])/3

                    weights.loc[patient_i, patient_j] = np.exp(-(self.df_dist.loc[patient_i, patient_j]**2/(eps*self.mu)))
                    weights.loc[patient_j, patient_i] = np.exp(-(self.df_dist.loc[patient_j, patient_i]**2/(eps*self.mu)))
                    
        return weights       
    
    def check_columns(self):
        scaler = MinMaxScaler()
        if 'label' in self.mirna.columns:
            self.mirna.drop(['label'], axis=1, inplace=True)
        if 'case_id' in self.mirna.columns:
            self.mirna.drop(['case_id'], axis=1, inplace=True)
            
        if 'label' in self.rna.columns:
            self.rna.drop(['label'], axis=1, inplace=True)
        if 'case_id' in self.rna.columns:
            self.rna.drop(['case_id'], axis=1, inplace=True)
            
        if 'label' in self.illumina.columns:
            self.illumina.drop(['label'], axis=1, inplace=True)
        if 'case_id' in self.illumina.columns:
            self.illumina.drop(['case_id'], axis=1, inplace=True)
            
        self.mirna = pd.DataFrame(scaler.fit_transform(self.mirna))
        self.rna = pd.DataFrame(scaler.fit_transform(self.rna))
        self.illumina = pd.DataFrame(scaler.fit_transform(self.illumina))

        return


    def find_k_neighbors(self, row, i, k=None): 
        row=deepcopy(row)
        #case of P matrix
        if k==None:
            del row[i]  #delete element of the same column of row index
            return row

        #case of S (find k elements with minimum distance value of W[i][j])
        else:
            k_neighbors_index=[]
            neigh = 0
            max_value = max(row)
            for j in range(0, len(row)):
                if j!=i:
                    min_index = row.index(min(row))
                    k_neighbors_index.append(min_index)
                    neigh+=1
                    row[min_index] = max_value
                    if neigh == k:
                        return k_neighbors_index


    def P_matrix(self, W, n_case_id, name):
        print('Calculating P matrix for {}...'.format(name))
        P=[]
        for i in tqdm(range(0, n_case_id)):
            row=[]
            for j in range(0,n_case_id):
                if i==j:
                    row.append(1/2)

                else:
                    k_neighbors = self.find_k_neighbors(W[i], i)
                    denominator = 2*sum(k_neighbors)
                    row.append(W[i][j]/denominator)
            P.append(row)
        print(np.array(P))
        return np.array(P)

    def S_matrix(self, W, n_case_id, name):
        print('Calculating S matrix for {}...'.format(name))
        S=[]
        for i in tqdm(range(0, n_case_id)):
            S_row=[]
            neighbors_indeces = self.find_k_neighbors(self.df_dist.iloc[i,:].to_numpy().tolist(), i, self.k)
            for j in range(0,n_case_id):
                if j not in neighbors_indeces:
                    S_row.append(0)

                else:
                    np_row = np.array(W[i])
                    denominator = sum(np_row[neighbors_indeces])
                    S_row.append(W[i][j]/denominator)
            S.append(S_row)
        print(np.array(S))
        return np.array(S)
    
    def product_matrix(self, S_matrix, P_matrix):
        result = np.dot(S_matrix, P_matrix)
        result = np.dot(result, S_matrix.T)
        return result
    
    def sum_matrix_P(self, P1, P2):
        return np.add(P1,P2)/2
    
    def fit(self, num_iter=None):
        if num_iter is not None:
            self.p_rna = self.starting_p_rna
            self.p_mirna = self.starting_p_mirna
            self.p_illumina = self.starting_p_illumina
            for i in range(0, num_iter):
                self.p_rna_t1 = self.product_matrix(self.s_rna, self.sum_matrix_P(self.p_mirna, self.p_illumina))
                self.p_mirna_t1 = self.product_matrix(self.s_mirna, self.sum_matrix_P(self.p_rna, self.p_illumina))
                self.p_illumina_t1 = self.product_matrix(self.s_illumina, self.sum_matrix_P(self.p_mirna, self.p_rna))
                print(self.p_rna_t1)
                print(self.p_mirna_t1)
                print(self.p_illumina_t1)
                self.p_rna = self.p_rna_t1
                self.p_mirna = self.p_mirna_t1
                self.p_illumina = self.p_illumina_t1
        else:
            print('ciao')

        return self
    
    def modified_fit(self, matrices_diff=None, max_iter=100):
        if matrices_diff is not None:
            self.p_rna = self.starting_p_rna
            self.p_mirna = self.starting_p_mirna
            self.p_illumina = self.starting_p_illumina
            for i in range(0, max_iter):
                self.p_rna_t1 = self.product_matrix(self.s_rna, self.sum_matrix_P(self.p_mirna, self.p_illumina))
                self.p_mirna_t1 = self.product_matrix(self.s_mirna, self.sum_matrix_P(self.p_rna, self.p_illumina))
                self.p_illumina_t1 = self.product_matrix(self.s_illumina, self.sum_matrix_P(self.p_mirna, self.p_rna))
                print(self.p_rna_t1)
                print(self.p_mirna_t1)
                print(self.p_illumina_t1)
                self.p_rna = self.p_rna_t1
                self.p_mirna = self.p_mirna_t1
                self.p_illumina = self.p_illumina_t1
                
                diff_matrix = np.abs(np.subtract(self.p_rna, self.p_mirna)) + np.abs(np.subtract(self.p_rna, self.p_illumina)) + np.abs(np.subtract(self.p_mirna, self.p_illumina))
                #diff_matrix= np.abs(np.mean(diff_matrix))
                if diff_matrix<=np.abs(matrices_diff):
                    print('number of iterations to reach difference: ', i)
                    break
                    
                if i == max_iter-1: ##impossible to reach matrices difference
                    print('impossible to reach indicated difference, try with a bigger difference value')
        else:
            print('no difference for matrices found')

        return self
    
    def clean(self):
        del self.p_rna
        del self.p_mirna
        del self.p_illumina
        
        del self.p_rna_t1
        del self.p_mirna_t1
        del self.p_illumina_t1
        
        
        del self.w_rna
        del self.w_mirna
        del self.w_illumina
        
        return self

In [23]:
df_mirna.sort_values(by='case_id', inplace=True)
df_rna.sort_values(by='case_id', inplace=True)
df_illumina.sort_values(by='case_id', inplace=True)

sm = SimilarityNetworkFusion(df_mirna.drop_duplicates(subset='case_id'),
                            df_rna.drop_duplicates(subset='case_id'),
                            df_illumina.drop_duplicates(subset='case_id'), k=100).calculate_matrix()

Calculating weights for RNA...
Calculating weights for miRNA...
Calculating weights for Illumina...
Calculating P matrix for RNA...
[[5.00000000e-01 5.38059625e-16 1.43713926e-16 ... 5.45904233e-15
  4.73343067e-28 3.29703178e-06]
 [1.29200810e-12 5.00000000e-01 1.89925577e-13 ... 1.48485898e-15
  3.30019218e-14 1.81773736e-18]
 [3.29241003e-11 1.81202291e-11 5.00000000e-01 ... 2.26883654e-13
  1.27351785e-24 1.07598270e-12]
 ...
 [1.07876004e-13 1.22196500e-17 1.95702572e-17 ... 5.00000000e-01
  1.47603290e-27 2.78694656e-17]
 [2.92732884e-17 8.49962744e-07 3.43783846e-19 ... 4.61937414e-18
  5.00000000e-01 4.56770793e-18]
 [1.04846349e-13 2.40727928e-29 1.49355219e-25 ... 4.48487588e-26
  2.34872955e-36 5.00000000e-01]]
Calculating P matrix for miRNA...
[[5.00000000e-01 1.63090220e-06 1.44113494e-03 ... 1.27786186e-03
  2.65991267e-07 1.38426830e-03]
 [2.33900874e-05 5.00000000e-01 4.15366257e-03 ... 1.02811795e-04
  1.38649576e-04 1.84574997e-04]
 [4.94117356e-04 9.93008001e-05 5.00

100%|██████████| 449/449 [00:04<00:00, 93.61it/s]


In [33]:
from scipy.spatial.distance import cdist

def modified_fit(sm, matrices_diff=None, max_iter=100):
        if matrices_diff is not None:
            sm.p_rna = sm.starting_p_rna
            sm.p_mirna = sm.starting_p_mirna
            sm.p_illumina = sm.starting_p_illumina
            for step in range(0, max_iter):
                sm.p_rna_t1 = sm.product_matrix(sm.s_rna, sm.sum_matrix_P(sm.p_mirna, sm.p_illumina))
                sm.p_mirna_t1 = sm.product_matrix(sm.s_mirna, sm.sum_matrix_P(sm.p_rna, sm.p_illumina))
                sm.p_illumina_t1 = sm.product_matrix(sm.s_illumina, sm.sum_matrix_P(sm.p_mirna, sm.p_rna))
               
                sm.p_rna = sm.p_rna_t1
                sm.p_mirna = sm.p_mirna_t1
                sm.p_illumina = sm.p_illumina_t1

                diff_matrix = 0
                for i in range(0, len(sm.p_rna)):
                    for j in range(i, len(sm.p_rna)):
                        diff_matrix += np.abs(sm.p_rna[i][j] - sm.p_mirna[i][j])
                        diff_matrix += np.abs(sm.p_illumina[i][j] - sm.p_mirna[i][j])
                        diff_matrix += np.abs(sm.p_illumina[i][j] - sm.p_rna[i][j])
                
                diff_matrix = diff_matrix**0.5
                print(step, ':', diff_matrix)
                
                #diff_matrix = np.abs(np.subtract(sm.p_rna, sm.p_mirna)) + np.abs(np.subtract(sm.p_rna, sm.p_illumina)) + np.abs(np.subtract(sm.p_mirna, sm.p_illumina))
                #diff_matrix= np.abs(np.mean(diff_matrix))
                if diff_matrix<=np.abs(matrices_diff):
                    print('number of iterations to reach difference: ', step)
                    break
                    
                if step == max_iter-1: ##impossible to reach matrices difference
                    print('impossible to reach indicated difference, try with a bigger difference value')
        else:
            print('no difference for matrices found')

        return sm

In [45]:
modified_fit(sm, matrices_diff=3)

0 : 24.616214374108893
1 : 17.738718104738822
2 : 15.013364509285433
3 : 12.651873011104975
4 : 11.280717926921973
5 : 10.215921341655836
6 : 9.418807965687215
7 : 8.767844963563164
8 : 8.2267244410242
9 : 7.763036226766368
10 : 7.359784435696586
11 : 7.003708090439803
12 : 6.686646021908226
13 : 6.403057083130548
14 : 6.147694060024382
15 : 5.916847548652389
16 : 5.707273887383818
17 : 5.51687115950872
18 : 5.343303839175281
19 : 5.184556840234193
20 : 5.039045905770055
21 : 4.90522234227554
22 : 4.781688795604559
23 : 4.667287186191647
24 : 4.561103123844948
25 : 4.462361250328064
26 : 4.370224833503684
27 : 4.284089048265311
28 : 4.203382284567481
29 : 4.12775459936376
30 : 4.056740444933242
31 : 3.989825571987176
32 : 3.9266698883457196
33 : 3.8669103496519353
34 : 3.8102959002650345
35 : 3.756638743746316
36 : 3.705665969345104
37 : 3.6572203954188818
38 : 3.6110734914745284
39 : 3.567020317722567
40 : 3.524894458859062
41 : 3.484609792666983
42 : 3.4460653694965413
43 : 3.4091070

In [ ]:
sm.fit(num_iter=50)

In [ ]:
import plotly.graph_objects as go

def plot_starting_graphs(df, y_pred):
    G = nx.from_numpy_matrix(df)
    print(nx.info(G))
    nx.draw_networkx(G, with_labels=False)
    return

In [ ]:
plot_starting_graphs(sm.p_rna, y_pred)

# Loading the label

In [46]:
y_illumina = LabelEncoder().fit_transform(df_illumina.drop_duplicates(subset='case_id').loc[:, 'label'].transform(lambda x: str(x)))
y_mirna = LabelEncoder().fit_transform(df_mirna.drop_duplicates(subset='case_id').loc[:, 'label'].transform(lambda x:  str(x)))
y_rna = LabelEncoder().fit_transform(df_rna.drop_duplicates(subset='case_id').loc[:, 'label'].transform(lambda x: str(x)))

In [47]:
y_pred = SpectralClustering(n_clusters=3, affinity='precomputed').fit(sm.p_mirna).labels_

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/manifold/_spectral_embedding.py:212: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


In [50]:
print('Rand Score:')
print('\tIllumina', adjusted_rand_score(y_illumina, y_pred))
print('\tMirna', adjusted_rand_score(y_mirna, y_pred))
print('\tRNA:', adjusted_rand_score(y_rna, y_pred))
print('\tmean:', (adjusted_rand_score(y_rna, y_pred) + adjusted_rand_score(y_illumina, y_pred) + adjusted_rand_score(y_mirna, y_pred))/3)

print('\n')
print('Silhouette score:')
print('\tIllumina', silhouette_score(sm.p_illumina, y_pred))
print('\tMirna', silhouette_score(sm.p_mirna, y_pred))
print('\tRNA:', silhouette_score(sm.p_rna, y_pred))

Rand Score:
	Illumina 0.8188601789932898
	Mirna 0.7970092411463793
	RNA: 0.7702807590377818
	mean: 0.7953833930591503


Silhouette score:
	Illumina 0.22158839680613623
	Mirna 0.22248530225416602
	RNA: 0.22158837971758452


In [ ]:
y_pred = KMeans(n_clusters=3).fit(sm.p_mirna).labels_
print('Rand Score:')
print('\tIllumina', adjusted_rand_score(y_illumina, y_pred))
print('\tMirna', adjusted_rand_score(y_mirna, y_pred))
print('\tRNA:', adjusted_rand_score(y_rna, y_pred))
print('\n')
print('Silhouette score:')
print('\tIllumina', silhouette_score(sm.p_illumina, y_pred))
print('\tMirna', silhouette_score(sm.p_mirna, y_pred))
print('\tRNA:', silhouette_score(sm.p_rna, y_pred))

In [ ]:
#####SI POTREBBE UTILIZZARE COME CONDIZIONE DI TERMINAZIONE CON UN CERTO VALORE
diff_matrix = np.abs(np.subtract(sm.p_rna, sm.p_mirna), sm.p_illumina)
print(np.mean(diff_matrix))